# HelioForge & CosmoView
## Overview, Tutorial, and Demonstration

This notebook provides a structured overview of the **HelioForge** simulation library
and the **CosmoView** visualization tool.


---
## 1. Conceptual overview

### HelioForge

HelioForge is a minimal heliocentric simulation framework based on a two-body
approximation.

Core characteristics:
- circular, coplanar orbits,
- explicit time stepping,
- deterministic behavior,
- clarity over physical completeness.

HelioForge provides:
- data models (`CentralBody`, `Planet`),
- system container (`SolarSystem`),
- time evolution (`Simulation`),
- Kepler helpers (`Kepler`),
- procedural generation and presets,
- JSON export and import.

### CosmoView

CosmoView is a pygame-based real-time viewer for HelioForge simulations.

It renders:
- orbital paths,
- planet motion,
- simulation time progression,

and provides interactive controls for navigation and speed.


---
## 2. Imports and setup


In [1]:

from helioforge import CentralBody, SolarSystem, make_solar_system
from helioforge.constants import AU_M, DAY_S
from helioforge.generation import generate_planets
from helioforge.kepler import Kepler
from helioforge.models import Planet
from helioforge.simulation import Simulation


---
## 3. Minimal generated system

This example demonstrates the simplest complete HelioForge workflow:
- create a star,
- generate planets procedurally,
- step the simulation,
- inspect positions.


In [2]:

star = CentralBody(
    name="DemoStar",
    mass_kg=1.0e30,
    radius_m=7.0e8,
    luminosity_w=0.0,
)

planets = generate_planets(star, n_planets=3, seed=1)
system = SolarSystem(star, planets)
sim = Simulation(system)

sim.step(DAY_S)
system.state_m()


{'Planet 1': (-4765018310.929516, 56575732603.16907),
 'Planet 2': (144953611397.21008, -580175218457.2361),
 'Planet 3': (-5147672402903.7705, 2313546322636.4463)}

### Time stepping

Simulation time advances explicitly. Positions update deterministically
based on orbital period and phase.


In [3]:

for day in range(3):
    sim.step(DAY_S)
    print(f"Day {day + 1}")
    for name, (x, y) in sim.system.state_m().items():
        print(f"  {name}: x={x:.2e} m, y={y:.2e} m")


Day 1
  Planet 1: x=-7.71e+09 m, y=5.63e+10 m
  Planet 2: x=1.46e+11 m, y=-5.80e+11 m
  Planet 3: x=-5.15e+12 m, y=2.31e+12 m
Day 2
  Planet 1: x=-1.06e+10 m, y=5.58e+10 m
  Planet 2: x=1.47e+11 m, y=-5.80e+11 m
  Planet 3: x=-5.15e+12 m, y=2.31e+12 m
Day 3
  Planet 1: x=-1.35e+10 m, y=5.51e+10 m
  Planet 2: x=1.48e+11 m, y=-5.80e+11 m
  Planet 3: x=-5.15e+12 m, y=2.31e+12 m


---
## 4. Solar System preset

HelioForge includes an approximate Solar System preset.
It provides a familiar reference system for demonstrations.


In [4]:

sun, planets_solar = make_solar_system()
system_solar = SolarSystem(sun, planets_solar)

system_solar.step(DAY_S)
[p.name for p in system_solar.planets]


['Mercury', 'Venus', 'Earth', 'Mars', 'Jupiter', 'Saturn', 'Uranus', 'Neptune']

---
## 5. Procedural generation in detail

Procedural generation allows control over:
- number of planets,
- orbital bounds,
- deterministic randomness.


In [5]:

planets_proc = generate_planets(
    star,
    n_planets=6,
    seed=42,
    inner_au=0.5,
    outer_au=25.0,
)

for p in planets_proc:
    print(f"{p.name:8} | {p.kind:10} | {p.distance_m / AU_M:.2f} AU")


Planet 1 | rocky      | 0.51 AU
Planet 2 | rocky      | 1.13 AU
Planet 3 | gas_giant  | 2.36 AU
Planet 4 | gas_giant  | 4.88 AU
Planet 5 | ice_giant  | 10.99 AU
Planet 6 | ice_giant  | 26.07 AU


---
## 6. Building a system manually

Systems can be constructed explicitly without generators.
This provides full control over orbital layout and parameters.


In [6]:

import math

star_hand = CentralBody(
    name="HandMadeStar",
    mass_kg=1.9885e30,
    radius_m=6.9634e8,
    luminosity_w=0.0,
)

k = Kepler(star_hand.mass_kg)

specs = [
    ("Alpha", "rocky",     1.0 * 5.972e24,   1.0 * 6.371e6,   0.7),
    ("Beta",  "rocky",     0.8 * 5.972e24,   0.9 * 6.371e6,   1.0),
    ("Gamma", "gas_giant", 0.6 * 1.898e27,   1.0 * 6.9911e7,  5.0),
]

planets_hand = []
for i, (name, kind, mass_kg, radius_m, distance_au) in enumerate(specs):
    distance_m = distance_au * AU_M
    planets_hand.append(
        Planet(
            name=name,
            kind=kind,
            mass_kg=mass_kg,
            radius_m=radius_m,
            distance_m=distance_m,
            phase_rad=i * 2.0 * math.pi / len(specs),
            period_s=k.period_s(distance_m),
            orbital_speed_mps=k.circular_speed_mps(distance_m),
        )
    )

system_hand = SolarSystem(star_hand, planets_hand)
sim_hand = Simulation(system_hand)

system_hand.state_m()


{'Alpha': (104718509490.0, 0.0),
 'Beta': (-74798935349.99997, 129555556378.25975),
 'Gamma': (-373994676750.0003, -647777781891.2985)}

In [7]:

for day in range(3):
    sim_hand.step(DAY_S)
    print(f"Day {day + 1}")
    for name, (x, y) in sim_hand.system.state_m().items():
        print(f"  {name}: x={x:.2e} m, y={y:.2e} m")


Day 1
  Alpha: x=1.05e+11 m, y=3.08e+09 m
  Beta: x=-7.70e+10 m, y=1.28e+11 m
  Gamma: x=-3.73e+11 m, y=-6.48e+11 m
Day 2
  Alpha: x=1.05e+11 m, y=6.15e+09 m
  Beta: x=-7.92e+10 m, y=1.27e+11 m
  Gamma: x=-3.72e+11 m, y=-6.49e+11 m
Day 3
  Alpha: x=1.04e+11 m, y=9.22e+09 m
  Beta: x=-8.14e+10 m, y=1.26e+11 m
  Gamma: x=-3.71e+11 m, y=-6.49e+11 m


---
## 7. Exporting and reloading systems

HelioForge systems can be serialized to JSON for reuse.


In [8]:

from helioforge.export import load_json, save_json

save_json(system_hand, "handmade_system.json")
system_loaded = load_json("handmade_system.json")

system_loaded.state_m()


{'Alpha': (104312212712.77792, 9215666485.468796),
 'Beta': (-81382416875.83553, 125524599746.11208),
 'Gamma': (-371000617773.4433, -649497201349.6836)}

---
## 8. Visualizing systems with CosmoView (Python)

CosmoView can render any HelioForge system directly from Python.


In [9]:

from cosmoview.app import run

run(
    width=1100,
    height=800,
    fps_limit=60,
    system=system_hand,
)


/home/lena/.helioforge/lib/python3.12/site-packages/pygame/pkgdata.py:25: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream, resource_exists


pygame 2.6.1 (SDL 2.28.4, Python 3.12.3)
Hello from the pygame community. https://www.pygame.org/contribute.html


---
## 9. CosmoView command-line usage

CosmoView can also be launched from a terminal.


```bash
# Built-in demos
cosmoview --mode solar
cosmoview --mode random

# Load a system exported from HelioForge
cosmoview --json handmade_system.json
```

---
## Summary

- HelioForge provides a deterministic heliocentric simulation model.
- Systems can be generated, handcrafted, exported, and reloaded.
- CosmoView visualizes any compatible HelioForge system interactively.
